In [76]:
import numpy as np
import pandas as pd
import collections
from math import sqrt,exp,pi
import operator

In [3]:
df = pd.read_csv("iris.csv")

In [4]:
df.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [5]:
data = df.to_numpy()
targets =np.unique(data[:,data.shape[1]-1])
for i in range(data.shape[0]):
    if  data[i,data.shape[1]-1] == targets[0]:
        data[i,data.shape[1]-1]  = 0.0
    elif data[i,data.shape[1]-1]  == targets[1]:
        data[i,data.shape[1]-1]  = 1.0
    else:
        data[i,data.shape[1]-1]  = 2.0
np.random.shuffle(data)

In [6]:
n= int(data.shape[0]*0.7)

In [7]:
data_train = data[:n,:]
data_test = data[n:,:]

In [8]:
train_features = data_train[:,:data.shape[1]-1]
train_target = data_train[:,data.shape[1]-1]

In [9]:
test_features = data_test[:,:data.shape[1]-1]
test_target = data_test[:,data.shape[1]-1]

In [197]:
class naive_bayes():
            
    def data_class(self,data):  # Creates a dict, keys are classes, values are arrays containing all the data in that class
        data_dict={}
        keys, counts = np.unique(data[:,data.shape[1]-1], return_counts=True)
        for e in keys:
            data_dict[e]=np.ones((counts[int(e)],data.shape[1]-1),float)
        ind_list=[-1]*(data.shape[1]-1)
        for i in range(data.shape[0]):
            ind_list[int(data[i,data.shape[1]-1])]= ind_list[int(data[i,data.shape[1]-1])] +1
            data_dict[data[i,data.shape[1]-1]][ind_list[int(data[i,data.shape[1]-1])],:]=data[i,:data.shape[1]-1]
        return data_dict

    
    
    def data_stats_class(self,data):# Creates a dict, keys are classes, values are lists, a list for each column containing [mean,std,len] per class
        data_dict=self.data_class(data)
        keys=list(data_dict.keys())
        stat_dict_class={}
        for e in keys:
            stat_dict_class[e] = [data_dict[e].shape[0]]
            for k in range((data_dict[e].shape[1])):
                stat_dict_class[e].append([np.mean(data_dict[e][:,k]),np.std(data_dict[e][:,k])])
        return stat_dict_class
    
    def gaussien(self,mean,stdev,x):            
        return (1 / (sqrt(2 * pi) * stdev)) * exp(-((x-mean)**2 / (2 * stdev**2 )))
    
    def proba_class(self,data,x):
        class_stats=self.data_stats_class(data)
        proba_class_dict= {}
        for class_value, class_summ in class_stats.items():
            proba_class_dict[class_value]=class_summ[0]/data.shape[0]
            for i in range(1,len(class_summ)):
                mean= class_summ[i][0]
                std = class_summ[i][1]
                proba_class_dict[class_value] = proba_class_dict[class_value] * self.gaussien(mean,std,x[i-1]) 
                # the gaussian in 37 returns P(feature/classs)
                #the proba_class_dict, has for keys classes , values P(c)*p(feature1/c)*...*p(featureN/c)
        return(proba_class_dict)
    
    
    def accuracy_metric(self,actual, predicted):
        correct = 0
        for i in range(len(actual)):
                if actual[i] == predicted[i]:
                    correct =correct+ 1
        return correct / float(len(actual)) * 100.0
    
    def predict(self,data,test_features,test_target):
        result = np.ones((test_target.shape),float)
        for i in range(test_features.shape[0]):
            result[i]=max(self.proba_class(data,test_features[i]).items(), key=operator.itemgetter(1))[0]
        accuracy =self.accuracy_metric(test_target,result)
        
        print("predicted =",result)
        print("accuracy = ", accuracy,"%")
          
            
                       
                    

In [194]:
model=naive_bayes()
model.predict(data_train,test_features,test_target)

predicted = [2. 1. 0. 2. 0. 0. 1. 2. 1. 0. 1. 0. 0. 1. 1. 2. 2. 2. 2. 0. 2. 1. 0. 1.
 2. 2. 2. 2. 1. 0. 1. 2. 1. 1. 1. 2. 1. 0. 1. 2. 1. 1. 2. 2. 1.]
accuracy =  95.55555555555556 %
